In [3]:
import pickle
import pandas as pd
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import sys

# Obtener la ruta del directorio base (nivel superior del proyecto)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(BASE_DIR, "src")

# Agregar SRC_PATH a sys.path si no está presente
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

In [5]:
with open(r'..\models\RandomForestClassifier.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)


In [6]:
df_oot = pd.read_csv(r'..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_oot.csv')
df_tr = pd.read_csv(r"..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_trtest.csv")
features = pd.read_csv(r'..\data\procesed\features.csv')

df_oot['id'] = df_oot['nit_enmascarado'].astype(str) + '#' + df_oot['num_oblig_orig_enmascarado'].astype(str) + '#' + df_oot['num_oblig_enmascarado'].astype(str)
df_tr['id'] = df_tr['nit_enmascarado'].astype(str) + '#' + df_tr['num_oblig_orig_enmascarado'].astype(str) + '#' + df_tr['num_oblig_enmascarado'].astype(str)

df_tr = df_tr.drop_duplicates(subset='id', keep='first')
df = pd.merge(df_oot, df_tr, on='id', how='left')

In [7]:
from data_engineer import FeatureSelector

limpieza = FeatureSelector(df, features)
df = limpieza.fit_transform()

Limpiando variables numéricas...
Separando X e y...
Detectando tipos de variables...
Preprocesando variables...
Transformando variables...
Reconstruyendo nombres de columnas...
Seleccionando variables...
Ajustando columnas faltantes...
Ordenando columnas...
Finalizando DataFrame...


In [12]:
# 2. Seleccionar columnas que usó el modelo
columnas_modelo = modelo.feature_names_in_
X = df[columnas_modelo]

# 3. Predecir
predicciones = modelo.predict(X)

# 4. Crear DataFrame final
df_submission = pd.DataFrame({
    'id': df['id'],
    'var_rpta_alt': predicciones,
    'var_rpta_alt_prob': modelo.predict_proba(X)[:, 1]
})

# 5. Guardar en CSV con el formato solicitado
df_submission.to_csv('submission.csv', index=False)


In [9]:
df_submission

,id,var_rpta_alt,var_rpta_alt_prob
0,257335#444821#635511,1,0.89
1,59584#350400#730364,1,0.89
2,397604#973821#106521,1,1.00
3,368086#382995#696856,1,1.00
4,255009#434238#645924,1,0.51
...,...,...,...
112544,316499#14503#1074624,1,0.96
112545,254311#30479#1013957,1,0.51
112546,483193#13732#1006779,1,0.89
112547,293824#22004#1062743,1,0.93


In [10]:
from predict import Predictor

oot_file = r'..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_oot.csv'
predictor = Predictor("RandomForestClassifier")
df_resultado = predictor.predecir(oot_path=oot_file)


🔧 Inicializando Predictor...
✅ Predictor inicializado correctamente.
🚀 Iniciando proceso de predicción...
📦 Cargando modelo: RandomForestClassifier.pkl...
✅ Modelo cargado con éxito.
📄 Cargando datos desde: ..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_oot.csv
🔗 Generando ID para merge y unificando datos...
🧼 Aplicando FeatureSelector...
Limpiando variables numéricas...
Separando X e y...
Detectando tipos de variables...
Preprocesando variables...
Transformando variables...
Reconstruyendo nombres de columnas...
Seleccionando variables...
Ajustando columnas faltantes...
Ordenando columnas...
Finalizando DataFrame...
✅ Datos cargados y limpiados.
🔮 Generando predicciones...
📊 Armando DataFrame de resultados...
✅ Predicción completada.


In [11]:
df_resultado

,id,var_rpta_alt,var_rpta_alt_prob
0,257335#444821#635511,1,0.89
1,59584#350400#730364,1,0.89
2,397604#973821#106521,1,1.00
3,368086#382995#696856,1,1.00
4,255009#434238#645924,1,0.51
...,...,...,...
112544,316499#14503#1074624,1,0.96
112545,254311#30479#1013957,1,0.51
112546,483193#13732#1006779,1,0.89
112547,293824#22004#1062743,1,0.93


In [11]:
df = pd.read_csv(r'D:\Desktop\juan_guzma_prueba_banco\data\raw\prueba_op_master_customer_data_enmascarado_completa.csv')

In [12]:
df

,nit_enmascarado,cod_tipo_doc,tipo_cli,ctrl_terc,genero_cli,ano_nac_cli,edad_cli,estado_civil,tipo_vivienda,num_hijos,...,nicho,region_of,nombre_dpto_dirp,egresos_mes,tot_patrimonio,ciiu,smmlv,year,month,ingestion_day
0,536377,1,PERSONA NATURAL,CLIENTE,F,1998.0,25.0,SOLTERO,FAMILIAR,0.0,...,MUJERES,CENTRO,SANTANDER,0.0,33750000.0,NaN,1160000.0,2023,12,31
1,512257,1,PERSONA NATURAL,CLIENTE,F,1984.0,38.0,UNION LIBRE,PROPIA,0.0,...,MUJERES,CARIBE,MAGDALENA,250000.0,5000000.0,ASALARIADOS,1160000.0,2023,9,30
2,5788,1,PERSONA NATURAL,CLIENTE SOCIAL,M,1960.0,63.0,NaN,NaN,0.0,...,NaN,DIRECCIÓN GENERAL,SIN INFORMACION,0.0,0.0,ASALARIADOS,1160000.0,2023,11,30
3,132245,1,PERSONA NATURAL,CLIENTE,F,1982.0,41.0,CASADO,NaN,1.0,...,MUJERES,SUR,VALLE,1000000.0,62513000.0,ASALARIADOS,1160000.0,2023,7,31
4,245279,1,PERSONA NATURAL,CLIENTE,M,1980.0,43.0,CASADO,ALQUILADA,0.0,...,NaN,BOGOTA Y CUNDINAMARCA,CALDAS,1000000.0,34135000.0,ASALARIADOS,1160000.0,2023,12,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429995,494311,1,PERSONA NATURAL,CLIENTE,F,1988.0,35.0,UNION LIBRE,NaN,0.0,...,NaN,CARIBE,CÓRDOBA,200000.0,25954000.0,OTRAS ACTIVIDADES DE SERVICIO DE APOYO A LAS E...,1160000.0,2023,12,31
429996,584402,1,PERSONA NATURAL,CLIENTE,F,1994.0,29.0,SOLTERO,FAMILIAR,0.0,...,INDEPENDIENTE,SUR,PUTUMAYO,300000.0,78155000.0,ASALARIADOS,1160000.0,2023,12,31
429997,362525,1,PERSONA NATURAL,CLIENTE,F,1998.0,25.0,UNION LIBRE,NaN,0.0,...,MUJERES,BOGOTA Y CUNDINAMARCA,BOGOT,350000.0,66000.0,ASALARIADOS,1160000.0,2023,10,31
429998,321589,1,PERSONA NATURAL,CLIENTE,M,2002.0,21.0,NO INFORMA,NaN,0.0,...,JOVENES,ANTIOQUIA,ANTIOQUIA,0.0,988483.0,NaN,1160000.0,2023,11,30


In [13]:
df.columns

Index(['nit_enmascarado', 'cod_tipo_doc', 'tipo_cli', 'ctrl_terc',
       'genero_cli', 'ano_nac_cli', 'edad_cli', 'estado_civil',
       'tipo_vivienda', 'num_hijos', 'personas_dependientes',
       'nivel_academico', 'ocup', 'act_econom', 'sector', 'subsector',
       'declarante', 'total_ing', 'tot_activos', 'tot_pasivos',
       'origen_fondos', 'f_vinc', 'f_ult_mantenimiento', 'canal_actualizacion',
       'cli_actualizado', 'segm', 'subsegm', 'nicho', 'region_of',
       'nombre_dpto_dirp', 'egresos_mes', 'tot_patrimonio', 'ciiu', 'smmlv',
       'year', 'month', 'ingestion_day'],
      dtype='object')